In [1]:
import os

import altair as alt

import pandas as pd
import numpy as np

import yaml

alphabet = 'ACGT'

def empirical_frequencies(counts):
    '''
    convert a nucleotide counts (dictionary {'A':3423, 'C':34543, ...}) to frequency
    '''
    total = sum(list(counts.values()))
    return np.array([counts[n]/total for n in alphabet])


def spectrum_to_matrix(spec):
    '''
    convert dictionary of mutation counts to mutation matrix
    '''
    M = np.zeros((4,4))
    for i1,n1 in enumerate(alphabet):
        for i2,n2 in enumerate(alphabet):
            if n1!=n2:
                M[i2,i1] = spec[f"{n1}to{n2}"]
    # normalize off-diagonal rates (just for standardization, doesn't affect the results)
    M /= M.sum()
    # will the diagonal with 'outflow' term to guarantee conservation of probability
    d = M.sum(axis=0)
    np.fill_diagonal(M,-d)

    return M


def equilibrium_probabilities(M):
    evals, evecs = np.linalg.eig(M)
    # find zero eigenvalue
    ii = np.argmin(np.abs(evals))
    assert np.abs(evals[ii])<1e-10
    # pull out corresponding eigenvector, return normalized to sum_i p_i = 1
    p = evecs[:,ii]
    return p/p.sum()

def get_ffsyn(ref):
    from Bio import SeqIO
    features  = [{'pos':i, 'nuc':n} for i,n in enumerate(ref.seq)]
    ## make a map of codon positions, ignore orf9b (overlaps N)
    for feat in ref.features:
        if feat.type=='CDS':
            gene_name = feat.qualifiers['gene'][0] if "gene" in feat.qualifiers else None
            if gene_name!='ORF9b':
                for gpos, pos in enumerate(feat.location):
                    tmp = {'pos_in_codon':(gpos%3)+1, 'gene':gene_name, 'four_fold':False}
                    if (gpos%3)+1==3:
                        codon12 = "".join(ref.seq[pos-2:pos])
                        if codon12 in ['TC','CT', 'CC', 'CG', 'AC', 'GT', 'GC','GG']:
                            tmp['four_fold'] = True
                    features[pos].update(tmp)

    return pd.DataFrame(features)

def get_sequence_from_genbank(accession):
    from Bio import SeqIO
    from Bio import Entrez
    Entrez.email = "richard.neher@unibas.ch"
    handle = Entrez.efetch(db="nucleotide", id=accession, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    return record

In [2]:
if os.path.isfile("config.yaml"):
    basedir = "./"
else:
    basedir = "../"
    
with open(os.path.join(basedir, "config.yaml")) as f:
    config = yaml.safe_load(f)
clade_synonyms = config["clade_synonyms"]

rates_by_clade =  pd.read_csv(os.path.join(basedir, "results/synonymous_mut_rates/rates_by_clade.csv"))

clades = rates_by_clade.clade.unique()

equilibrium_probabilities_by_clade = {}
for clade in clades:
    subset = rates_by_clade.loc[rates_by_clade.clade==clade]
    tmp_spectrum = {}
    for rate in subset.itertuples():
        tmp_spectrum[rate.mut_type] = rate.rate

    equilibrium_probabilities_by_clade[clade] = equilibrium_probabilities(spectrum_to_matrix(tmp_spectrum))

eqprob_df = (
    pd.DataFrame.from_dict(
        equilibrium_probabilities_by_clade,
        orient="index",
        columns=list(alphabet)
    )
    .applymap(lambda x: x.real)
    .rename_axis("virus")
    .reset_index()
    .assign(virus=lambda x: x["virus"] + " (" + x["virus"].map(clade_synonyms) + ")")
)

eqprob_df

,virus,A,C,G,T
0,20A (B.1),0.162199,0.072122,0.027598,0.738081
1,20B (B.1.1),0.167663,0.069790,0.028297,0.734250
2,20C (B.1.367),0.177134,0.069814,0.030583,0.722470
3,20E (B.1.177),0.165593,0.066128,0.036844,0.731436
4,20G (B.1.2),0.178689,0.063486,0.031226,0.726599
5,20I (Alpha),0.177130,0.063275,0.035565,0.724030
6,21C (Epsilon),0.185620,0.065432,0.034616,0.714331
7,21I (Delta),0.142835,0.080941,0.030449,0.745775
8,21J (Delta),0.144964,0.084301,0.030591,0.740144
9,21K (Omicron BA.1),0.248047,0.075088,0.061554,0.615312


In [3]:
accessions = {
    "MN908947": "SARS-CoV-2",
    "MN996532.2": "RaTG13",
    "MZ937000.1": "BANAL-52",
    "KY352407.1": "BtKY72",
    "AY278488.2 ": "SARS-CoV-1",
}

empirical_frequencies_references = {}
for accession in accessions:
    print(f"Parsing {accession=}")
    ref = get_sequence_from_genbank(accession)
    ref_genome = get_ffsyn(ref)
    empirical_frequencies_references[accession] = empirical_frequencies({i:r for i,r in ref_genome.loc[ref_genome.four_fold & (~ref_genome.four_fold.isna())].nuc.value_counts().iteritems()})

empirical_freqs_df = (
    pd.DataFrame.from_dict(
        empirical_frequencies_references,
        orient="index",
        columns=list(alphabet)
    )
    .rename_axis("virus")
    .reset_index()
    .assign(virus=lambda x: x["virus"].map(accessions) + " (" + x["virus"] + ")")
)

empirical_freqs_df

Parsing accession='MN908947'
Parsing accession='MN996532.2'
Parsing accession='MZ937000.1'
Parsing accession='KY352407.1'
Parsing accession='AY278488.2 '


,virus,A,C,G,T
0,SARS-CoV-2 (MN908947),0.289660,0.136922,0.064920,0.508499
1,RaTG13 (MN996532.2),0.292148,0.134402,0.063900,0.509550
2,BANAL-52 (MZ937000.1),0.288941,0.133176,0.066118,0.511765
3,BtKY72 (KY352407.1),0.262702,0.152661,0.092222,0.492415
4,SARS-CoV-1 (AY278488.2 ),0.286147,0.166511,0.089552,0.457789


In [4]:
df = pd.concat(
    [
        eqprob_df.assign(group="predicted equilibrium frequencies"),
        empirical_freqs_df.assign(group="virus frequencies at 4-fold degenerate sites"),
    ]
).melt(
    id_vars=["virus", "group"],
    var_name="nucleotide",
    value_name="frequency",
)

chart = (
    alt.Chart(df)
    .encode(
        x=alt.X("frequency", title="nucleotide frequency"),
        y=alt.Y("virus", title=None, sort=empirical_freqs_df["virus"].unique().tolist()),
        shape=alt.Shape("nucleotide"),
        fill=alt.Fill("nucleotide"),
        facet=alt.Facet("group", columns=1, title=None),
        tooltip=df.columns.tolist(),
    )
    .mark_point(stroke=None, size=60, opacity=1)
    .resolve_scale(y="independent")
    .properties(width=250, height=alt.Step(15))
)

outfile = os.path.join(basedir, "results/equilibrium_freqs/equilibrium_freqs.html")
os.makedirs(os.path.dirname(outfile), exist_ok=True)
chart.save(outfile)

chart

alt.Chart(...)